# The Hangman Game
### Using Tensorflow version 0.10. Latest version is 1.2. Some functions differ slightly in the latest version.

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# allowing better python 2 & python 3 compatibility 
from __future__ import print_function 

def hangman(secret_word, guesser, max_mistakes=8, verbose=True):
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            try:
                guessed.add(guess)
            except:
                print(guessed,guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

def human(mask, guessed):
    print('Enter your guess:')
    return raw_input().lower().strip()
    #return input().lower().strip() # swap with above for python 3


## Loading the dataset

In [3]:
f = open('train.txt').read().splitlines()

# Randomly permutes the words
f = [f[i] for i in np.random.permutation(len(f))]
max_len = np.max([len(x) for x in f])
test = []
train = []
for i,item in enumerate(f):
        train.append(item)
        
test = []
with open("test.txt", mode="r") as myFile:
    for line in myFile:
        try:
            test.append(line.strip())
        except:
            pass
    

# Creating the Neural Network

In [4]:
initializer = tf.contrib.layers.xavier_initializer()
lstm_dim = 300
model_weight_file = "model.weights"
load_model = False
batch_size = 64

In [5]:
def fclayer(inp, num_out, act=tf.nn.tanh, name=None, reuse=None):
    if name is None:
        name = "fclinear"
    with tf.variable_scope(name, reuse=reuse):
        shape = inp.get_shape()
        num_in = int(shape[1])
        W = tf.get_variable(
            name="W", shape=[num_in, num_out], initializer=initializer)
        b = tf.get_variable(
            name="b", shape=[num_out], initializer=tf.constant_initializer(0))

        out = tf.matmul(inp, W) + b
        return out


def fclayer3d(inp, num_out, act=tf.nn.tanh, name=None, reuse=None):
    if name is None:
        name = "fclinear3d"
    with tf.variable_scope(name, reuse=reuse):
        shape = inp.get_shape()
        d = int(shape[2])
        l = int(shape[1])
        inp = tf.reshape(inp, [-1, d])
        out = fclayer(inp=inp, num_out=num_out, act=act)
        out = tf.reshape(out, [-1, l, num_out])
        return out

In [6]:
# Defining the input variables
input_seq = tf.placeholder(
            dtype=tf.int32,
            shape=[None, max_len],
            name="input_seq"
        )  # [x, l]

seq_len = tf.placeholder(
            dtype=tf.int32,
            shape=[None],
            name="seq_len"
        )  # [x]

labels = tf.placeholder(
            dtype=tf.int32,
            shape=[None, 26],
            name="labels"
        )  # [x, 26]

# One hot encoding of the input
one_hot_seq = tf.one_hot(input_seq, depth=26)  # [x, l, 26]

# Creating the LSTM

In [7]:
lstm_inp = fclayer3d(one_hot_seq, lstm_dim)  # [x, l, lstm_dim]

cell_fw = tf.contrib.rnn_cell.LSTMCell(
            lstm_dim,
            initializer=initializer,
            state_is_tuple=True,
            activation=tf.nn.tanh
        )

cell_bw = tf.contrib.rnn_cell.LSTMCell(
            lstm_dim,
            initializer=initializer,
            state_is_tuple=True,
            activation=tf.nn.tanh
        )

lstm_inp = tf.transpose(lstm_inp, [1, 0, 2])
lstm_inp = tf.unpack(lstm_inp)

outputs, state_fw, state_bw = tf.nn.bidirectional_rnn(
            cell_fw=cell_fw,
            cell_bw=cell_bw,
            inputs=lstm_inp,
            dtype=tf.float32,
            sequence_length=seq_len
        )

output_state = tf.pack(state_fw,state_bw)
output_state = tf.transpose(output_state, [1, 0, 2])  # [x, 4, d]
output_state = tf.reshape(output_state, [-1, 4 * lstm_dim])  # [x, 4*d]

AttributeError: 'module' object has no attribute 'rnn_cell'

# Using LSTM output to predict

In [ ]:
pred_layer1 = fclayer(output_state, 500)
pred_layer2 = fclayer3d(pred_layer1, 500)
pred_out = fclayer3d(pred_layer2, 26)  # [x, 26]
pred_probs = tf.nn.softmax(pred_out)

# Creating loss function and optimizer

In [ ]:
log_prob = tf.log(pred_probs)  # [x, 26]
loss = - log_prob * labels  # [x, 26]
loss = tf.reduce_sum(loss, axis=1)  # [x]
loss = tf.reduce_mean(loss)  # scalar

optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
init = tf.initialize_all_variables()
saver = tf.train.Saver()

# Train

In [ ]:
def get_batches(data):
    num_batches = len(data) / batch_size
    for i in range(num_batches):
        batch_x = data[batch_size * i: batch_size * (i+1)]
        len_x = np.zeros(batch_size)
        batch_y = np.zeros((batch_size, 26))
        
        # TODO for each example in batch_x:
        # - Map each character to a number between 0 to 25.
        # - Randomly replace some numbers with -1 (-1 denotes '_'). Note: replace all occurences of the numbers.
        # - For each replaced number i, set batch_y[example, i] = number of occurrences of i.
        # - Compute the length of each example and store in the len_x.
        # - Append '-1' to all examples so that their length is equal to max_len
        
        yield batch_x, len_x, batch_y

In [ ]:
for batch_x, len_x, batch_y in get_batches(train):
    print batch_x
    print len_x
    print batch_y
    break  # breaking early for testing

In [ ]:
with tf.Session as sess:
    if load_model:
        saver.restore(sess, model_weight_file)
        # If loading trained model, no need to train again
    else:
        sess.run(init)
        # TODO:
        # - Iterate over the batches and train the model
        # - Calculate the train and test accuracy after regular intervals to track your training
        # - Save your model after regular intervals

In [ ]:
def guesser(mask, guessed):
    # TODO:
    # - Define your guesser function so that it uses the trained model to guess characters.
    # - Return the character with the highest probability (or highest logit) and which has not been guessed yet.
    # - To get the logits from the model, you have to do the same kind of one-hot mapping as done in
    #   get_batches function.

In [8]:
import string
test = "aello"
test_int = [0]*len(test)
i = 0
for char in test:
    test_int[i] = ord(char)-97
    i += 1
    
num_to_replace = np.random.choice(test_int)
ind = test_int.index(num_to_replace)
count = 0
while ind :
    test_int[ind] = -1
    count += 1
    try:
        ind = test_int.index(num_to_replace)
    except:
        print(num_to_replace,count)
        break
print(num_to_replace)
ex_len = len(test_int)
test_int = test_int + [-1]*(max_len - len(test_int))
print(test_int)

11 2
11
[0, 4, -1, -1, 14, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
